# Before Building Model

In [ ]:
from IPython.display import clear_output
!pip3 install pytorch-tabnet
clear_output()

import pandas as pd
import numpy as np
import os
import random
import warnings
warnings.simplefilter('ignore')

from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetClassifier
import torch

TRAIN_PATH = "../input/tabular-playground-series-feb-2022/train.csv"
TEST_PATH = "../input/tabular-playground-series-feb-2022/test.csv"
SAMPLE_SUBMISSION_PATH = "../input/tabular-playground-series-feb-2022/sample_submission.csv"
SUBMISSION_PATH = "submission.csv"

ID = "row_id"
TARGET = "target"

SEED = 2022
def seed_everything(seed=SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything()

VAL_SIZE = 0.3
MAX_EPOCHS = 200
MODEL_PATIENCE = 10
MODEL_EVAL_MERIC = 'accuracy'
MODEL_BATCH_SIZE = 64
MODEL_VIRTUAL_BATCH_SIZE = 32
MODEL_VERBOSE = 10

In [ ]:
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

train = reduce_memory_usage(train)
test = reduce_memory_usage(test)

train = train.drop([ID],axis=1)
test = test.drop([ID],axis=1)

In [ ]:
def toLabelEncode(TRAIN_PATH,TEST_PATH):
    train = pd.read_csv(TRAIN_PATH)
    test = pd.read_csv(TEST_PATH)

    cat_col = train.describe(include="O").columns.tolist()
    
    for col in cat_col:
        if col == TARGET: continue
        
        train[col] = train[col].astype("category").cat.codes
        test[col] = test[col].astype("category").cat.codes
        
    return train,test

train,test = toLabelEncode(TRAIN_PATH,TEST_PATH)

def checkNull_fillData(df):
    for col in df.columns:
        if len(df.loc[df[col].isnull() == True]) != 0:
            if df[col].dtype == "float64" or df[col].dtype == "int64":
                df.loc[df[col].isnull() == True,col] = df[col].mean()
            else:
                df.loc[df[col].isnull() == True,col] = df[col].mode()[0]
                
checkNull_fillData(train)
checkNull_fillData(test)

# Build Model

In [ ]:
features = test.columns

X = train.drop([ID,TARGET],axis=1).to_numpy()
y= train[TARGET].to_numpy().squeeze()
X_test = test.drop([ID],axis=1).to_numpy()

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=VAL_SIZE) 

model = TabNetClassifier(verbose=MODEL_VERBOSE,seed=SEED)
model.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=[MODEL_EVAL_MERIC],
    max_epochs=MAX_EPOCHS,
    patience=MODEL_PATIENCE,
    batch_size=MODEL_BATCH_SIZE, 
)

# After Building Model

In [ ]:
sub = pd.read_csv(SAMPLE_SUBMISSION_PATH)
sub[TARGET] = model.predict(X_test)
sub.to_csv(SUBMISSION_PATH,index=False)
sub.head()
